# Problem Set 3: Merging and regular expressions

**Total points (without extra credit)**: 44

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset3_inputdata` (need to unzip): 

- `jobs`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


## Resources from class

- [Lecture](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/06_qss20_w23_mergereshape.pdf) and [activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/03_reshaping_merging_solutions.ipynb) on exact merging
- [Lecture](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/07_qss20_w23_regex.pdf) and [activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/04_regex_blank.ipynb) on regular expressions (both coming in class on Feb. 1)
- DataCamp modules on both of these

# 0. Load packages & data (1 point total)

In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## 0.1 Load data (0 points)

Load the following dataset stored in `pset3_inputdata`&mdash;`debar.csv`&mdash;storing it as a dataframe named `debar`. This represents employers temporarily banned from hiring workers.

View the `head()` and columns of this dataframe.

In [2]:
## your code here
debar = pd.read_csv("debar.csv")
debar.head()

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


The most notable column names mean as follows:
- `Name`:	Company name of agricultural employer
- `City, State`:	City and state where employer located
- `Violation`:	Type of program violation
- `Start date`:	Start date of debarment (temporary ban)

## 0.2 Inspect data (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [3]:
## your code here

# # of rows in debar
nrows = debar.shape[0]

# # of unique employer names
unique = debar.Name.nunique()

print(nrows, unique)
# there are more rows than number of unique employer names in the dataframe.
# therefore, there are multiple rows for some employers.

114 98


# 1. Reshape data and check duplicates (19 points total)

## 1.1 Make indicator for violation number (2 points)

To make it possible to reshape data, make an indicator for the violation number for each business. The indicator should take the value of `viol` if it's the first row/potential violation, `viol2` if the second row/potential violation, etc.

**Hint:**
- One way to do this is by using an if-else statement to check whether the business name is the same as in the row above (assuming rows are ordered by name). Grouping by employer name and checking the number of unique offenses would also work.

In [4]:
## your code here
debar.sort_values(by='Name')

# Add a new column 'violation_num' -- indicator for the violation num for each business
debar['violation_num'] = debar.groupby('Name').cumcount() + 1

# Rename violation num so it takes the value of 'viol', 'viol2, etc
debar['violation_num'] = debar['violation_num'].apply(lambda x: f"viol{x}" if x > 1 else "viol")

debar


,Name,"City, State",Violation,Duration,Start date,End date
94,69 Farms LLC,"Preston, Idaho",Failure to respond to audit request,2 years,9/24/2016,9/23/2018
97,AB Ranch,"Stephenville, Texas",Failure to respond to audit request,2 years,4/28/2017,4/27/2019
102,Abe- Nancy Froese,"Seminole, TX",Failure to respond to audit request,2 years,11/23/2017,11/23/2019
36,Agecy I LLC,"Idabel, OK",Impeding the Audit Process – Non- Response,2 years,8/27/2015,8/26/2017
53,"Agecy II, LLC","Ipswich, SD",Impeding the Audit Process – Non- Response,2 years,3/25/2015,3/24/2017
...,...,...,...,...,...,...
73,Xavier Horne,"Lyons, Georgia",Non-payment of certification fee,1 year,6/16/2016,6/15/2017
89,Xavier Horne,"Lyons, Georgia",Failure to respond to audit request,2 years,9/27/2017,9/26/2019
88,Yesenia Perez,"Axson, Georgia",Failure to respond to audit request,2 years,9/27/2017,9/26/2019
9,Yolanda Chavez,"Santa Maria, CA",Non Payment,1 year,7/23/2014,7/22/2015


,Name,"City, State",Violation,Duration,Start date,End date,violation_num
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016,viol
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016,viol
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015,viol
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,viol
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,viol
...,...,...,...,...,...,...,...
109,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2 years,2/9/2018,2/8/2020,viol2
110,Jesus Ledesma,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,viol
111,Turner Farms,"Healy, KS",Failure to comply with the employer's obligati...,7 months,7/17/19,2/10/20,viol2
112,B & R Harvesting and Paul Cruz (individual),"Collins, Georgia",WHD Debarment,3 years,4/9/17,4/9/20,viol


## 1.2 Clean up state names (3 points)

Inspect the state names in the business violation data. Which states have names sometimes indicated in long format vs. two-letter abbreviation, e.g. "New Hampshire" vs. "NH"? Which of these may have more than one violation?

**Hint:** 
- One way to do this is to extract state names from the `City, State` column using regex and/or string methods.
- The simplest way to check if a state may have more than one violation is to check how many times they appear using `value_counts()`. 

In [5]:
# your code here

# Create new 'state' and 'city' columns from the 'City, State' column of the debar dataframe
debar['State'] = debar['City, State'].str.split(', ').str[-1].str.strip()
debar['City'] = debar['City, State'].str.split(', ').str[0].str.strip()
debar['State'] = debar['State'].astype(str)

# Inspecting the state column, there is a "nan" state at index 39.  
debar.loc[debar['State'] == 'nan'] 
# Dropping this row
debar = debar.drop(debar.index[39])

# Create a list of unique state names
unique_states = debar['State'].unique().tolist()

# Check if a state name is in long format
def is_long_format(state_name):
    return len(state_name) > 2

# Filter out state names that are in long format
long_format_states = [state for state in unique_states if is_long_format(state)]

# Check if a state has more than one violation
multiple_violations = debar['State'].value_counts()[debar['State'].value_counts() > 1].index.tolist()


print("States with long format names:" + str(long_format_states))
print("States with multiple violations:" + str(multiple_violations))

debar

,Name,"City, State",Violation,Duration,Start date,End date,violation_num,State,City
39,Valley View Orchards,NaN,Impeding the Audit Process – Partial Response;,2 years,7/22/2015,7/21/2017,viol,nan,NaN


States with long format names:['Kansas', 'Texas', 'Utah', 'Florida', 'California', 'Georgia', 'North Dakota', 'Arkansas', 'Vermont', 'Tennessee', 'Louisiana', 'North Carolina', 'Idaho', 'Massachusetts']
States with multiple violations:['TX', 'GA', 'KS', 'Georgia', 'FL', 'KY', 'SD', 'UT', 'Texas', 'ND', 'NY', 'North Dakota', 'CA', 'CO', 'Florida', 'AR', 'Utah', 'OK', 'LA', 'MA', 'MT']


,Name,"City, State",Violation,Duration,Start date,End date,violation_num,State,City
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016,viol,ND,Leads
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016,viol,SD,Selby
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015,viol,NY,Pulaski
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,viol,KS,Dighton
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,viol,KS,Greensburg
...,...,...,...,...,...,...,...,...,...
109,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2 years,2/9/2018,2/8/2020,viol2,TX,Mount Vernon
110,Jesus Ledesma,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,viol,FL,Mulberry
111,Turner Farms,"Healy, KS",Failure to comply with the employer's obligati...,7 months,7/17/19,2/10/20,viol2,KS,Healy
112,B & R Harvesting and Paul Cruz (individual),"Collins, Georgia",WHD Debarment,3 years,4/9/17,4/9/20,viol,Georgia,Collins


To make the state names consistent with the jobs data below, convert any discrepant state names to the two-letter abbreviation format. 

**Hints:**
- You could do this for all state names (to be safe) or only those you just identified. 
- If you want to change ANY discrepant state names from long format to two-letter format (i.e., a complete conversion), you can load state names from a complete name/abbreviation crosswalk and use that to change state names. Here is code to load in such a list ([from this blog](https://towardsdatascience.com/state-name-to-state-abbreviation-crosswalks-6936250976c)):
```python
crosswalk_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
pd.read_html(crosswalk_url)[0]
```

In [6]:
## your code here
# Load state names crosswalk
crosswalk_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
state_names = pd.read_html(crosswalk_url)[0]
state_names = dict(zip(state_names['Description'], 
                          state_names['Code']))

# Convert state names to two-letter abbreviation
def convert_state_name(state_name):
    state_abbr = state_names.get(state_name)
    return state_abbr

debar['State'] = debar['State'].apply(lambda x: convert_state_name(x) if x in long_format_states else x)
debar['City, State'] = debar['City'].str.cat(debar['State'], sep=", ")


## cleaning city names for Loewen Harvesting LLC and Maple Ridge Custom Services, LLC
debar['City, State'].replace('Brownsville, TX', 'Brownfield, TX', inplace=True)
debar['City, State'].replace('Altheimer, AK', 'Altheimer, AR', inplace=True)

debar = debar.drop(columns=['City', 'State'])
debar


,Name,"City, State",Violation,Duration,Start date,End date,violation_num
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016,viol
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016,viol
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015,viol
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,viol
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,viol
...,...,...,...,...,...,...,...
109,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2 years,2/9/2018,2/8/2020,viol2
110,Jesus Ledesma,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,viol
111,Turner Farms,"Healy, KS",Failure to comply with the employer's obligati...,7 months,7/17/19,2/10/20,viol2
112,B & R Harvesting and Paul Cruz (individual),"Collins, GA",WHD Debarment,3 years,4/9/17,4/9/20,viol


## 1.3 Investigate duplicated rows (4 points)

A. Create a new column in `debar`, `is_repeated`, that tells us whether an employer (`Name`) is repeated > 1 times.

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using `value_counts()` on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret with at least a sentence. If you notice any cases of duplicate business names where 'City, State' does NOT match exactly, check these manually and make them consistent.

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [7]:
## your code here
# A. Create a new column in debar, is_repeated, that tells us whether an employer (Name) is repeated > 1 times. 
debar['is_repeated'] = debar['Name'].isin(debar['Name'].value_counts()[debar['Name'].value_counts() > 1].index).astype(int)
repeated_rows = debar[debar["is_repeated"] == True]

# B. Print the rows where 'is_repeated == True'
repeated_rows
## intepretation: It seems like the state name of the repeated rows are in long-format version in 1.2, I suspect this could the reason of duplicated data.

# check if City, State match and make them consistent
grouped = debar.groupby('Name')['City, State'].nunique()
duplicate_names = grouped[grouped > 1].index.tolist()

# Filter data for duplicate names where 'City, State' does NOT match exactly
duplicate_debar = debar[debar['Name'].isin(duplicate_names)]
inconsistent_debar = duplicate_debar.groupby('Name')['City, State'].apply(lambda x: x.nunique() > 1)
inconsistent_debar = inconsistent_debar[inconsistent_debar == True].reset_index()

for name, group in inconsistent_debar.groupby('Name'):
    group_values = group['City, State'].tolist()
    if len(group_values) > 0:
        new_value = group_values[0] # choose the first value as the new consistent value
        debar.loc[debar['City, State'] == new_value]

# C. Subset to the rows where is_repeated == True and save that data as mult_debar
mult_debar = debar[debar["is_repeated"] == True].copy()

        
# Print the head and shape of mult_debar
mult_debar.head()
mult_debar.shape
mult_debar





,Name,"City, State",Violation,Duration,Start date,End date,violation_num,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol,1
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol,1
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,viol,1
11,Loewen Harvesting LLC,"Brownfield, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,viol,1
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,viol,1
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,viol,1
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,viol,1
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,viol,1
17,"Maple Ridge Custom Services, LLC","Altheimer, AR",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,viol,1
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,viol,1


/var/folders/0x/rl39d06x18d0p7tvgzg1rq340000gn/T/ipykernel_27534/2874338955.py:16: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  inconsistent_debar = duplicate_debar.groupby('Name')['City, State'].apply(lambda x: x.nunique() > 1)


,Name,"City, State",Violation,Duration,Start date,End date,violation_num,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol,1
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol,1
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,viol,1
11,Loewen Harvesting LLC,"Brownfield, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,viol,1
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,viol,1


(32, 8)

,Name,"City, State",Violation,Duration,Start date,End date,violation_num,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol,1
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol,1
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,viol,1
11,Loewen Harvesting LLC,"Brownfield, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,viol,1
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,viol,1
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,viol,1
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,viol,1
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,viol,1
17,"Maple Ridge Custom Services, LLC","Altheimer, AR",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,viol,1
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,viol,1


## 1.4 Reshape `mult_debar` to wide format to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in these data: e.g., converting `Violation` to lowercase or replacing spelled-out states with two-dig state codes.

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

For a successful reshaping, make sure each row in `mult_debar` shows only a single business (no duplicates under `Name`) and isn't missing either `start_date_viol1` or `start_date_viol2`.

Show the contents of `mult_debar` and its shape.

In [8]:
## your code here
mult_debar_wide = mult_debar.pivot_table(index=['Name', 'City, State'], values='Start date', columns='violation_num', aggfunc='first')

mult_debar_wide.columns = ['start_date_viol1', 'start_date_viol2']
mult_debar_wide
mult_debar_wide.shape


,,start_date_viol1,start_date_viol2
Name,"City, State",,
Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014
Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014
"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014
Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015
Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018
F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014
Loewen Harvesting LLC,"Brownfield, TX",8/20/2014,8/20/2014
Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015
"Maple Ridge Custom Services, LLC","Altheimer, AR",11/16/2014,11/16/2014


(16, 2)

## 1.5 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning, `debar`, filter as follows:
- For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
- For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
- Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [9]:
# A. Add column is_dup to mult_debar_wide
mult_debar_wide['is_dup'] = mult_debar_wide['start_date_viol1'] == mult_debar_wide['start_date_viol2']
mult_debar_wide = mult_debar_wide.reset_index()
mult_debar_wide

,Name,"City, State",start_date_viol1,start_date_viol2,is_dup
0,Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014,True
1,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014,True
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014,True
3,Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015,False
4,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018,True
5,F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014,True
6,Loewen Harvesting LLC,"Brownfield, TX",8/20/2014,8/20/2014,True
7,Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015,True
8,"Maple Ridge Custom Services, LLC","Altheimer, AR",11/16/2014,11/16/2014,True
9,Mark Duncan,"Roosevelt, UT",11/16/2014,11/16/2014,True


In [10]:
# B. Filter the original debar dataframe
debar_true = mult_debar_wide[mult_debar_wide['is_dup'] == True]
true_list = list(debar_true.Name)

debar_duplicates = debar[debar['Name'].isin(true_list) & (debar['violation_num'] == 'viol')]
debar_nonduplicates = debar[~debar['Name'].isin(true_list)]


debar_clean = pd.concat([debar_duplicates, debar_nonduplicates], axis=0)
debar_clean.drop(columns = 'is_repeated', axis=1, inplace=True)
debar_clean.reset_index(drop=True, inplace=True)
debar_clean


# testing if it keeps all the violation numbers for the non-duplicates
debar_clean.loc[debar_clean['Name'] == 'Xavier Horne'] 
print("debar_clean shape:", debar_clean.shape)
print("# of unique employer names:", debar_clean['Name'].nunique())

,Name,"City, State",Violation,Duration,Start date,End date,violation_num
0,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol
1,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,viol
3,Loewen Harvesting LLC,"Brownfield, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,viol
4,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,viol
...,...,...,...,...,...,...,...
95,Walker Place,"Danville, IL",Failure to comply with the employer's obligati...,2 months,11/19/2019,1/26/2020,viol
96,County Fair Farm (company) and Andrew Williams...,"Jefferson, ME",WHD Debarment,3 years,3/8/2017,3/8/2020,viol
97,Jesus Ledesma,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,viol
98,B & R Harvesting and Paul Cruz (individual),"Collins, GA",WHD Debarment,3 years,4/9/17,4/9/20,viol


,Name,"City, State",Violation,Duration,Start date,End date,violation_num
63,Xavier Horne,"Lyons, GA",Non-payment of certification fee,1 year,6/16/2016,6/15/2017,viol
79,Xavier Horne,"Lyons, GA",Failure to respond to audit request,2 years,9/27/2017,9/26/2019,viol2


debar_clean shape: (100, 7)
# of unique employer names: 97


# 2. Merging and regex (19 points total)



## 2.1 Load data on job postings (1 point)

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations. Since most of the bans have expired, let's see which of those employers posted new H-2A jobs in the first quarter of 2021.

First, load the `jobs.csv` data stored in `pset3_inputdata`.

In [11]:
# your code here to load the data 
jobs = pd.read_csv('jobs.csv')
jobs.head()


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"Fazio Farms Operating Company, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,Charlie Sunderland,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Michael Rudebusch,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Lodahl Farms,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"Dunson Harvesting, Inc.","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4


The most notable column names mean as follows:
- `CASE_NUMBER`:	Administrative identifier for an employer's H-2A visa application
- `EMPLOYER_NAME`:	Employer name
- `EMPLOYER_CITY`:	Employer city
- `EMPLOYER_STATE`:	Employer state
- `EMPLOYER_ADDRESS_1`:	Employer address (only need to use if doing the geocoding extra credit)

##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [12]:
## your code here
inner = pd.merge(jobs, debar_clean, how="inner", left_on="EMPLOYER_NAME", right_on="Name")

print(inner)

          CASE_NUMBER                           CASE_STATUS  \
0  H-300-20287-876656  Determination Issued - Certification   

             RECEIVED_DATE            DECISION_DATE  \
0  2020-10-20 09:20:32.010  2020-11-09 00:00:00.000   

  TYPE_OF_EMPLOYER_APPLICATION H2A_LABOR_CONTRACTOR NATURE_OF_TEMPORARY_NEED  \
0          Individual Employer                    Y                 Seasonal   

  EMERGENCY_FILING   EMPLOYER_NAME TRADE_NAME_DBA  ... EMAIL_TO_APPLY  \
0                Y  Rafael Barajas            NaN  ...            NaN   

                WEBSITE_TO_APPLY TOTAL_ADDENDUM_A_RECORDS            Name  \
0  https://seasonaljobs.dol.gov/                        7  Rafael Barajas   

   City, State                         Violation Duration  Start date  \
0  Sebring, FL  Non-payment of certification fee   1 year   9/23/2016   

    End date  violation_num  
0  9/22/2017           viol  

[1 rows x 145 columns]


## 2.3 Targeted regex (11 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Convert to upper (3 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [13]:
## insert your code to turn into uppercase here
jobs_upper = [x.upper() for x in jobs.EMPLOYER_NAME]

debar_upper = [x.upper() for x in debar_clean.Name]


In [14]:
## insert your code for the random sample
print(random.sample(jobs_upper, 15))
print(random.sample(debar_upper, 15))

['KUNAFIN LLC', 'COLORFUL GARDENS WHOLESALE LLC', 'BEARSKIN FARM', 'BEEROM APIARIES, INC.', 'WESTERN RANGE ASSOCIATION', 'ARCH FRINK LLC', 'RALPH HOWARD', 'TRIANGLE SPRAYING, INC', 'CAJUN GATORS INC', 'KERMIT KUETHE', 'PACE BROTHERS', 'RAMBO NURSERY LLC', 'HAGYARD-DAVIDSON-MCGEE ASSOCIATES PLLC', 'EL ARROZ FARMS, LLC', "DAN & JERRY'S GREENHOUSE, INC. - IN"]
['OLD TREE FARMS/VERPAALEN CUSTOM SERVICE', 'LOEWEN HARVESTING LLC', 'LOUIE M. ASUMENDI', 'PETER PETERS', 'GLENDA HARROD FARMS', 'DOVE CREEK FARMS', 'REIMER’S INC.', 'LEONARD SMITH FARMS', 'DAVID JACKSON', 'RB FARM PICKING, INC', 'LESLIE RENEE DREW', 'DEAL FAMILY FARM', 'MARK DUNCAN', 'M KENESTON CORP', 'SHARON MATHIS']


In [15]:
## insert your code for assigning the uppercase names back to the data
jobs["EMPLOYER_NAME"] = jobs_upper
debar_clean["Name"] = debar_upper

### 2.3.2 Clean up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [16]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [17]:
## insert your code here with the regex pattern for part A
pattern = r"(?<=INC)\.|(?<=CO)\.|(?<=LLC)\."

## insert your code to use re.sub to apply the pattern to the test cases for part B

fixed_pos_1 = re.sub(pattern, "", pos_example_1)
fixed_pos_2 = re.sub(pattern, "", pos_example_2)
fixed_neg = re.sub(pattern, "", neg_example)

print(fixed_pos_1, fixed_pos_2, fixed_neg)

CISCO PRODUCE INC AVOYELLES HONEY CO, LLC E.V. RANCH LLP


### 2.3.3 Clean employer names (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [18]:
## your code here to clean the columns
jobs_clean_names = [re.sub(pattern, "", x) for x in jobs.EMPLOYER_NAME]
debar_clean_names = [re.sub(pattern, "", x) for x in debar_clean.Name]

jobs["name_clean"] = jobs_clean_names
debar_clean["name_clean"] = debar_clean_names

In [19]:
## your code here to print the head
jobs[jobs.EMPLOYER_NAME != jobs.name_clean][["EMPLOYER_NAME", "name_clean"]].head()
debar_clean[debar_clean.Name != debar_clean.name_clean][["Name", "name_clean"]].head()

,EMPLOYER_NAME,name_clean
4,"DUNSON HARVESTING, INC.","DUNSON HARVESTING, INC"
7,"FARM LABOR ASSOCIATION FOR GROWERS, INC.","FARM LABOR ASSOCIATION FOR GROWERS, INC"
14,"MCLAIN FARMS, INC.","MCLAIN FARMS, INC"
17,"BONNIE PLANTS, INC.","BONNIE PLANTS, INC"
18,"B & W QUALITY GROWERS, INC.","B & W QUALITY GROWERS, INC"


,Name,name_clean
16,ANTON FERTILIZER INC.,ANTON FERTILIZER INC
17,"GREAT PLAINS FLUID SERVICE, INC.","GREAT PLAINS FLUID SERVICE, INC"
18,PROMAX INC.,PROMAX INC
21,REIMER'S INC.,REIMER'S INC
22,CISCO PRODUCE INC.,CISCO PRODUCE INC


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe. Write a sentence describing how name cleaning affected the match results.

**Note**: this manual cleaning process is inefficient and may miss other likely matches. A better approach would be fuzzy matching, which would recognize that 'Slash EV ranch' is a highly similar string to 'slash ev ranch llp' and match them without us needing to use regex to make the strings identical. We may talk about this in class later if time!

In [20]:
## your code here
## A
inner2 = pd.merge(jobs, debar_clean, how="inner", on="name_clean")
inner2

## B
debar_clean["name_clean_2"] = [re.sub(r"(?<=E)\.|(?<=V)\.|\sLLP", "", name) for name in debar_clean.name_clean]

## C
left = pd.merge(jobs, debar_clean, how="left", left_on="name_clean", right_on="name_clean_2", indicator=True)

left[left._merge=="both"]

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,name_clean,Name,"City, State",Violation,Duration,Start date,End date,violation_num
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,https://seasonaljobs.dol.gov/,7,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, FL",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,viol


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,Name,"City, State",Violation,Duration,Start date,End date,violation_num,name_clean_y,name_clean_2,_merge
791,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,RAFAEL BARAJAS,"Sebring, FL",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,viol,RAFAEL BARAJAS,RAFAEL BARAJAS,both
1115,H-300-20306-894148,Determination Issued - Certification,2020-11-02 18:11:29.140,2020-11-24 00:00:00.000,Individual Employer,N,Seasonal,N,SLASH EV RANCH,NaN,...,SLASH E.V. RANCH LLP,"Rifle, CO",WHD Debarment,1 year,11/15/2014,11/14/2015,viol,SLASH E.V. RANCH LLP,SLASH EV RANCH,both


Name cleaning did increase the matches, but only from one match to two. Because the different names are so irregular, it is hard to clean them completely without writing a lot of regex over many steps. 

# 3. Regex to separate companies from individuals (6 points)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [21]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern 
pattern = r'(.*)\s\(COMPANY\)\sAND\s(.*)\s\(INDIVIDUAL\)'

# testing on the pos_example
re.findall(pattern, pos_example)
match = re.search(pattern, pos_example)
match.group(1) #company name
match.group(2) #individual name
# tesing on the neg_example
re.findall(pattern, neg_example)

[('COUNTY FAIR FARM', 'ANDREW WILLIAMSON')]

'COUNTY FAIR FARM'

'ANDREW WILLIAMSON'

[]

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [22]:
# your code here

def name_search(name):
    match = re.search(pattern, name)
    if match:
        return match.group(1), match.group(2)
    else:
        return name, name
    
# Apply the function to the "name_clean" column and create two new columns in the debar_clean dataframe
debar_clean[["co_name", "ind_name"]] = debar_clean["name_clean"].apply(name_search).apply(pd.Series)

# Fill in full name_clean string if no mathc found
debar_clean["co_name"].fillna(debar_clean["name_clean"], inplace=True)
debar_clean["ind_name"].fillna(debar_clean["name_clean"], inplace=True)



   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [23]:
# Get the rows for the positive and negative examples
neg_example = debar_clean[debar_clean["name_clean"] == "CISCO PRODUCE INC"]
pos_example = debar_clean[debar_clean["name_clean"] == "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"]
debar_regex = pd.concat([pos_example, neg_example], axis=0)

debar_regex[['name_clean','co_name','ind_name','Violation']]


,name_clean,co_name,ind_name,Violation
96,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM,ANDREW WILLIAMSON,WHD Debarment
22,CISCO PRODUCE INC,CISCO PRODUCE INC,CISCO PRODUCE INC,Failure to respond to audit (no response)
51,CISCO PRODUCE INC,CISCO PRODUCE INC,CISCO PRODUCE INC,Impeding the Audit Process – Non- Response


# 4. Optional extra credit: Geospatial visualization (2 points)

Geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on a map of Georgia. This involves Googling and using external sources to figure out the code (a common practice in real-life data science), since we haven't spatial data in the course. 

**Hints:**
- Relevant columns include `EMPLOYER_ADDRESS_1` 
- The geocoding might have a long runtime, so feel free to implement it in a separate `.py` script that you submit alongside your notebook and to just read in the geocoded data

**Resources:**
- [Discussion of geocoding addresses -> lat/long](https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/)
- [Discussion of plotting lat/long dots against a map using geopandas](https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73)

In [24]:
## your code here
import urllib
import requests

coords = jobs[['EMPLOYER_ADDRESS_1']]

url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(coords["EMPLOYER_ADDRESS_1"][0]) +'?format=json'
response = requests.get(url).json()
print('Latitude: '+response[0]['lat']+', Longitude: '+response[0]['lon'])

Latitude: 45.58369865, Longitude: -122.65261644502496


In [25]:
def geocode2(locality):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(locality) +'?format=json'
    response = requests.get(url).json()
    if(len(response)!=0):
        return(response[0]['lat'], response[0]['lon'])
    else:
        return('-1')

# sample 10 to decrease runtime and test if the code works
coords['final'] = coords['EMPLOYER_ADDRESS_1'].sample(n=10).apply(geocode2)

/var/folders/0x/rl39d06x18d0p7tvgzg1rq340000gn/T/ipykernel_27534/4001210917.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords['final'] = coords['EMPLOYER_ADDRESS_1'].sample(n=10).apply(geocode2)


In [26]:
map_data = coords[~coords.final.isna()]
map_data

,EMPLOYER_ADDRESS_1,final
146,4581 Buckhorn Drive,"(44.7412647181251, -85.67223486673787)"
182,524 Plant Farm Road,"(31.0762859, -83.394542)"
183,2305 Cypress Lane,"(41.54362813590044, -88.13883637938498)"
324,25201 S Louisiana Hwy 82,-1
1031,170 Red Hawk Road,"(38.044341125814846, -122.5393198262528)"
1081,PO Box 21,"(32.8893373, -117.1336005)"
1515,3082 Walnut Hill Road,"(37.9318454, -84.45114345931191)"
2200,3726 County Street 2964,"(35.2245713, -97.716115)"
2207,143 Mary Irene Lane,"(35.253006, -91.777721)"
2277,2540 Highway 31,"(49.6974988, -116.9151153)"


In [27]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import fiona
import pyogrio


georgia = gpd.read_file('./tl_2019_13_cousub/tl_2019_13_cousub.shp')

crs = {'init':'EPSG:4326'}
geometry = [Point(xy) for xy in zip(coords['long'], coords['lat'])]
geo_df = gpd.GeoDataFrame(df, 
                          crs = crs, 
                          geometry = geometry)

fig, ax = plt.subplots(figsize = (10,10))
georgia.to_crs(epsg=4326).plot(ax=ax, color='lightgrey')
geo_df.plot(ax=ax)
ax.set_title('Georgia Ag Companies')

DriverError: ./tl_2019_13_cousub/tl_2019_13_cousub.shp: No such file or directory